In [19]:
!pip install pyspark

from pyspark.sql import SparkSession


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
spark = SparkSession.builder.master("local[*]").getOrCreate() #Seteaza numarul de core-uri de utilizat (* = toate)

csv_path = "/content/Students/StudentsPerformance/" #Locatia datelor noastre

df = spark.read.csv(csv_path, header=True, inferSchema=True) #Incarcarea datelor din CSV intr-un dataframe Spark

In [21]:
table_name = "my_table" #definirea unei variabile table_name
df.createOrReplaceTempView(table_name) #crearea unui view temporar
query = "SELECT * FROM my_table" #definirea unei interogari SQL
result = spark.sql(query) #rularea interogarii
result.show() #afisarea rezultatelor

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [22]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [23]:
df.describe().show()


+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null|        null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null|        null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

In [24]:
df = df.withColumnRenamed('parental level of education', 'parentEducation')
df = df.withColumnRenamed('test preparation course', 'preparationCourse')
df = df.withColumnRenamed('math score', 'mathScore')
df = df.withColumnRenamed('reading score', 'readingScore')
df = df.withColumnRenamed('writing score', 'writingScore')
df.describe().show()
df.write.format('csv').mode('overwrite').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted')


+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|         mathScore|      readingScore|     writingScore|
+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|  count|  1000|          1000|              1000|        1000|             1000|              1000|              1000|             1000|
|   mean|  null|          null|              null|        null|             null|            66.089|            69.169|           68.054|
| stddev|  null|          null|              null|        null|             null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|associate's degree|free/reduced|        completed|                 0|                17|               10|
|    max|  male|       group E|  s

In [25]:
csv_path = "/content/Students/StudentsPerformanceFormatted" #salvam locatia csv-ului formatat intr-o noua variabila
df = spark.read.csv(csv_path, header=True, inferSchema=True) #incarcam noile date
from pyspark.sql.functions import col, round, format_number, when #importam lucrul cu coloane

df = df.withColumn('finalScore', (col('mathScore') + col('readingScore') + col('writingScore'))/3) #mai adaugam o coloana care determina media generala
df = df.withColumn('finalScore', format_number(col('finalScore'), 2)) #afisam doar 2 decimale in aceasta noua coloana
df = df.withColumn('admited', when(col('finalScore') > 50 , True).otherwise(False))#mai adaugam o coloana care va afisa true pentru cazul in care studentul a fost admis(finalScore > 50) sau false pentru cazul in care nu a fost admis(finalScore < 50)
df.show()#afisam rezultatele
#Salvam dataframe-ul modificat 
df.write.mode('overwrite').format('csv').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted2')

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 

In [26]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parentEducation: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- preparationCourse: string (nullable = true)
 |-- mathScore: integer (nullable = true)
 |-- readingScore: integer (nullable = true)
 |-- writingScore: integer (nullable = true)
 |-- finalScore: string (nullable = true)
 |-- admited: boolean (nullable = false)



In [27]:

 
#salvam locatia csv-ului formatat intr-o noua variabila
csv_path = "/content/Students/StudentsPerformanceFormatted2"

#incarcam noile date
df = spark.read.csv(csv_path, header=True, inferSchema=True) 

df.show()

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 

SPARK SQL

1.Sa se afiseze toti studenții care nu au obținut o nota de trecere la matematica ordonand de la cea mai mica nota

In [28]:
df.createOrReplaceTempView("students")

result = spark.sql("SELECT * FROM students WHERE mathScore < 50 ORDER BY mathScore ASC")

result.show()


+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group C|  some high school|free/reduced|             none|        0|          17|          10|       9.0|  false|
|female|       group B|       high school|free/reduced|             none|        8|          24|          23|     18.33|  false|
|female|       group B|  some high school|free/reduced|             none|       18|          32|          28|      26.0|  false|
|female|       group B|      some college|    standard|             none|       19|          38|          32|     29.67|  false|
|female|       group C|      some college|free/reduced|             none|       22|          39| 

2.Sa se afiseze studentii care nu au fost admiși și nivelul de pregătire al părinților

In [29]:
result = spark.sql("SELECT admited, parentEducation FROM students WHERE admited == 'false' ORDER BY parentEducation")

result.show()


+-------+------------------+
|admited|   parentEducation|
+-------+------------------+
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
+-------+------------------+
only showing top 20 rows



3. Sa se afiseze numarul de studenti care au fost admisi

In [30]:
result = spark.sql("SELECT COUNT(*) AS TotalAdmisi FROM students WHERE admited == 'true'")

result.show()


+-----------+
|TotalAdmisi|
+-----------+
|        880|
+-----------+



4. Sa se afiseze numarul total de studenti respinsi

In [31]:
result = spark.sql("SELECT COUNT(*) AS TotalRespinsi FROM students WHERE admited == 'false'")

result.show()


+-------------+
|TotalRespinsi|
+-------------+
|          120|
+-------------+



5. Sa se afișeze studenții care nu au fost admiși dar, totuși, au urmat cursul de pregătire 

In [42]:
result = spark.sql("SELECT * FROM students WHERE admited == 'false' AND preparationCourse == 'completed' ")

result.show()


+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|  male|       group C|   master's degree|free/reduced|        completed|       46|          42|          46|     44.67|  false|
|  male|       group A|      some college|free/reduced|        completed|       50|          47|          54|     50.33|  false|
|  male|       group C|associate's degree|free/reduced|        completed|       43|          45|          50|      46.0|  false|
|  male|       group B|       high school|    standard|        completed|       60|          44|          47|     50.33|  false|
|  male|       group C|  some high school|free/reduced|        completed|       53|          37| 

6. Sa se afiseze nota medie la matematica pe fiecare gen:

In [62]:
result = spark.sql("SELECT gender, FORMAT_NUMBER(AVG(mathScore), 2) AS NotaMedieLaMatematica FROM students GROUP BY gender")
result.show()



+------+---------------------+
|gender|NotaMedieLaMatematica|
+------+---------------------+
|female|                63.63|
|  male|                68.73|
+------+---------------------+



7. Sa se calculeze nota medie la citit pe fiecare rasa/etnicitate

In [63]:
result = spark.sql("SELECT `race/ethnicity`, FORMAT_NUMBER(AVG(readingScore), 2) AS NotaMedieLaCitit FROM students GROUP BY `race/ethnicity` ")
result.show()

+--------------+----------------+
|race/ethnicity|NotaMedieLaCitit|
+--------------+----------------+
|       group B|           67.35|
|       group C|           69.10|
|       group D|           70.03|
|       group A|           64.67|
|       group E|           73.03|
+--------------+----------------+



8. Sa se gaseasca numeral total de students pentru fiecare nivel de educatie al parintilor

In [77]:
result = spark.sql("SELECT parentEducation, COUNT(*) AS student_count FROM students GROUP BY parentEducation")
result.show()

+------------------+-------------+
|   parentEducation|student_count|
+------------------+-------------+
|  some high school|          179|
|associate's degree|          222|
|       high school|          196|
| bachelor's degree|          118|
|   master's degree|           59|
|      some college|          226|
+------------------+-------------+



9. Sa se afiseze numeral de studenti admisi si procentajul pentru fiecare tip de prânz

In [72]:
result = spark.sql("SELECT lunch,  COUNT(*) AS totalStudents, SUM(CASE WHEN admited = 'Yes' THEN 1 ELSE 0 END) AS studentiAdmisi, FORMAT_NUMBER((SUM(CASE WHEN admited = 'Yes' THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS procentajAdmisi FROM students GROUP BY lunch")
result.show()

+------------+-------------+--------------+---------------+
|       lunch|totalStudents|studentiAdmisi|procentajAdmisi|
+------------+-------------+--------------+---------------+
|free/reduced|          355|           283|          79.72|
|    standard|          645|           597|          92.56|
+------------+-------------+--------------+---------------+



10. Sa se afiseze nota medie la scris pe gen pentru studentii care au completat cursul de pregatire

In [76]:
result = spark.sql("SELECT gender, FORMAT_NUMBER(AVG(writingScore), 2) AS notaMedieLaScris FROM students WHERE preparationCourse = 'completed' GROUP BY gender")
result.show()

+------+----------------+
|gender|notaMedieLaScris|
+------+----------------+
|female|           78.79|
|  male|           69.79|
+------+----------------+

